Cell 1 — Setup & Inputs (Miami)

In [2]:
# --- Setup imports & helpers ---
from agents.owner_parser_agent import invoke_owner_parser_agent
from agents.user_parser_agent import invoke_user_parser_agent
from agents.matching_agent import match_for_new_user, match_for_new_owner
from utils.qdrant_connection import client , SIM_COLLECTION , USER_COLLECTION , OWNER_COLLECTION
from qdrant_client.models import PayloadSchemaType
from qdrant_client.http import exceptions as qexc


def ensure_similarity_indexes():
    """Ensure similarity_collection has the payload indexes needed for filtering."""
    for field, schema in [
        ("owner_id", PayloadSchemaType.KEYWORD),
        ("user_id",  PayloadSchemaType.KEYWORD),
    ]:
        try:
            client.create_payload_index(
                collection_name=SIM_COLLECTION,
                field_name=field,
                field_schema=schema,
            )
            print(f"✅ Index ready on '{field}'")
        except qexc.UnexpectedResponse as e:
            if "already exists" in str(e).lower():
                print(f"ℹ️ Index on '{field}' already exists")
            else:
                raise

# --- Miami test inputs ---
owner_input = """
Bright studio in Manhattan, NY, $2,200/month.
Near Central Park, quiet street, doorman building.
Move-in available in September. No smoking.
""".strip()

user_input = """
Need a small apartment in Brooklyn with quick access to the subway.
Max budget $2,500/month, moving in September. Prefer quiet building.
""".strip()

print("Inputs ready.")


Inputs ready.


Parse & Upload (Owner + User)

In [3]:
print("➕ Uploading owner listing…")
try:
    owner_point_id = invoke_owner_parser_agent(owner_input)
    print("Owner point id:", owner_point_id)
except Exception as e:
    print("❌ Error uploading owner listing:", e)
    owner_point_id = None


print("\n➕ Uploading user query…")
try:
    user_point_id = invoke_user_parser_agent(user_input)
    print("User point id:", user_point_id)
except Exception as e:
    print("❌ Error uploading user query:", e)
    user_point_id = None

# Make sure similarity_collection has indexes (in case it was rebuilt)
ensure_similarity_indexes()


➕ Uploading owner listing…
✅ Upserted 1 points into 'owner_agent_listings' without resetting the collection.
✅ Uploaded owner listing with ID: e1802ea7b1c4e15eb6443331e17c9087 to 'owner_agent_listings'
✅ Uploaded profile to owner_profiles: Robert Wilson
Owner point id: e1802ea7b1c4e15eb6443331e17c9087

➕ Uploading user query…
✅ Upserted 1 points into 'user_agent_listings' without resetting the collection.
✅ Uploaded user query with ID: 7702862d70cc91336d157488df3f5000 to 'user_agent_listings'
✅ Uploaded profile to user_profiles: Thomas Smith
User point id: 7702862d70cc91336d157488df3f5000
✅ Index ready on 'owner_id'
✅ Index ready on 'user_id'


In [4]:
from utils.qdrant_connection import pretty_print_listing
pretty_print_listing("Owner listing", OWNER_COLLECTION, owner_point_id)
pretty_print_listing("User listing",  USER_COLLECTION, user_point_id)


🔎 Owner listing (collection='owner_agent_listings', id='e1802ea7b1c4e15eb6443331e17c9087')
  ✅ Found point:
{
  "listing_id": "e1802ea7b1c4e15eb6443331e17c9087",
  "state": [
    "New York"
  ],
  "picture_url": null,
  "price": 2200,
  "bedrooms": 1,
  "available_from": "September",
  "soft_attributes": "near Central Park, quiet street, doorman building, no smoking"
}

🔎 User listing (collection='user_agent_listings', id='7702862d70cc91336d157488df3f5000')
  ✅ Found point:
{
  "listing_id": "7702862d70cc91336d157488df3f5000",
  "state": [
    "New York"
  ],
  "picture_url": null,
  "price": 2500,
  "bedrooms": null,
  "available_from": "September",
  "soft_attributes": "quick access to subway, quiet building",
  "source": "user_query"
}


Match Both Directions & Show Details

In [ ]:
print("\n🔎 Matching for NEW USER → owner listings …")
user_matches = match_for_new_user(user_point_id)

print("🔎 Matching for NEW OWNER → user queries …")
owner_matches = match_for_new_owner(owner_point_id)

from utils.qdrant_connection import print_owner_matches_with_details, print_user_matches_with_details

print_user_matches_with_details(user_matches, top_k=5)
print_owner_matches_with_details(owner_matches, top_k=5)



🔎 Matching for NEW USER → owner listings …
🔎 Matching for NEW OWNER → user queries …
Top 5 owners for this user:
  score=0.6277  owner_id=25086c6a-863a-1ad7-d451-5f6adcb4a268
    [state: New York | price: 1200 | bedrooms: 1 | available_from: None] soft: remodeled, ground-floor unit, clean, well-maintained living space, shared kitchen and bathroom, close to subway
  score=0.6260  owner_id=099901dd-41fc-750e-d336-99fdcc8e6bfc
    [state: New York | price: 2500 | bedrooms: 1 | available_from: None] soft: quiet space, close to subway and attractions, access to kitchen, pet-friendly, cat on premises
  score=0.6081  owner_id=58492549-b50a-33f8-8126-b8770e436741
    [state: New York | price: 2500 | bedrooms: 1 | available_from: None] soft: charming location at Union Square and East Village, well-maintained unit, close to subway, great shopping nearby
  score=0.6079  owner_id=2ac09842-934c-a73a-e092-f72f4c6d8910
    [state: New York | price: 2000 | bedrooms: 1 | available_from: None] soft: co

[{'user_id': 'e55b6e68-2c54-bd81-e4d6-8f494e9a944f',
  'owner_id': 'e1802ea7-b1c4-e15e-b644-3331e17c9087',
  'score': 0.27391103,
  'filter_used': {'collection': 'user_agent_listings',
   'state': ['New York'],
   'price_owner': 2200,
   'bedrooms_owner': 1,
   'available_from': 'September'},
  'timestamp': 1755260133.7234905}]

Profiles (Owner & User)

In [6]:
from utils.qdrant_connection import get_user_profile, get_owner_profile

owner_profile = get_owner_profile(owner_point_id)  # dict or None
user_profile  = get_user_profile(user_point_id)    # dict or None

print("Owner profile:", owner_profile)
print("User profile:", user_profile)


Owner profile: {'profile_id': 'e1802ea7b1c4e15eb6443331e17c9087', 'type': 'owner', 'full_name': 'Robert Wilson', 'email': 'user5@example.com', 'phone': '+1-555-2310', 'application_date': '2025-08-11', 'number_of_shows': '0'}
User profile: {'profile_id': '7702862d70cc91336d157488df3f5000', 'type': 'user', 'full_name': 'Thomas Smith', 'email': 'user2@example.com', 'phone': '+1-555-9937', 'application_date': '2025-07-16', 'number_of_shows': '0'}


Quick Estimated Summaries

In [7]:
from agents.matching_agent import summarize_estimated_for_user, summarize_estimated_for_owner

print("\n=== Estimated summary for USER ===")
summarize_estimated_for_user(user_point_id, user_matches, check_top_k=5)

print("\n=== Estimated summary for OWNER ===")
summarize_estimated_for_owner(owner_point_id, owner_matches, check_top_k=5)



=== Estimated summary for USER ===
=== Estimated opportunities for you ===
- You appear as the #1 candidate in ~22 listing(s).
- You appear in the top 5 for ~42 listing(s).
- You have a strong ‘hard-attribute’ fit with ~15 listing(s).
- Total listings evaluated in this preview: 50
- Your current best score: 0.6277 (owner_id=25086c6a-863a-1ad7-d451-5f6adcb4a268)

Note: These are early estimates based on current matches.
Final invitations depend on scheduling, fairness (giving chances to those with fewer shows),
and listing popularity. You may not be invited to all matched properties.


=== Estimated summary for OWNER ===
=== Estimated demand for your listing ===
- Your listing appears as the #1 match for ~0 user(s).
- Your listing appears in the top 5 for ~0 user(s).
- There are ~1 user(s) whose requirements strongly fit your listing.
- Total users evaluated in this preview: 1
- Best current candidate score: 0.2739 (user_id=e55b6e68-2c54-bd81-e4d6-8f494e9a944f)

Note: These are early e

Trigger LLM Showing Decisions (saves CSV)

In [1]:
from utils.qdrant_connection import summarize_shows_text
from agents.manage_showings_agent import run_daily_decisions

filtered_results, csv_path = run_daily_decisions(max_invites=3)
summary_text = summarize_shows_text(filtered_results)
print(summary_text)
# In Streamlit:
# st.code(summary_text)



ManageShowings:   0%|          | 0/3 [00:00<?, ?owner/s]

✅ Done. 2 owners processed (all show == 1). CSV → logs/showings_decisions_20250816-113108.csv
there will be 2 shows that we will schedule for you :

show 1 :
owner: Olivia Miller, +1-555-2859 , user7@example.com, Chicago, 1 bed rooms , 2100 dollars, have been shown 0 times. have been applicant to the system at 2025-07-15..
an invitaion mail will be sent to you, the owner and the 4 matched users.

show 2 :
owner: David Thomas, +1-555-8036 , user6@example.com, Miami, 1 bed rooms , 2100 dollars, have been shown 0 times. have been applicant to the system at 2025-07-19..
an invitaion mail will be sent to you, the owner and the 4 matched users.



 Apply Decisions to Dataset (Dry Run first)

In [9]:
# If you already added organize_dataset_after_showings to utils, import it.
# Otherwise, define it earlier or copy in the implementation we discussed.
try:
    from utils.qdrant_connection import organize_dataset_after_showings
except ImportError:
    raise RuntimeError("organize_dataset_after_showings not found. Make sure it's available in utils.qdrant_connection")

# Dry run (no writes) to preview effects
summary_preview = organize_dataset_after_showings(results, top_k_for_recovery=10, dry_run=False)
print(summary_preview)

# If everything looks good, actually apply:
# summary_applied = organize_dataset_after_showings(results, top_k_for_recovery=10, dry_run=False)
# print(summary_applied)


Applying showing decisions:   0%|          | 0/20 [00:00<?, ?owner/s]


=== organize_dataset_after_showings summary ===
owners_processed:   20
owners_with_show:   16
owners_incremented: 16
users_incremented:  66
pairs_deleted:      66
{'owners_processed': 20, 'owners_with_show': 16, 'owners_incremented': 16, 'users_incremented': 66, 'pairs_deleted': 66, 'updated_owner_ids': ['f4e5df6b-fcf6-565a-853c-52b526e5218f', '95eb7b7d-310c-eb7d-e7eb-7decad6bd592', 'dfa26407-aa3f-d174-c22d-5ce8740fdee5', '295861f9-fe39-648b-46ee-c4c6d849b5c1', '443105c9-5329-eb90-4d8d-228353664992', '96bf7189-814a-f0cb-3f2d-989cee5918d1', '4d73959b-1743-d711-7e3b-918a60418470', '45fe06ed-634e-2da4-1d22-e3e4656e8081', 'fab31824-dcfa-58b7-4aec-19a962769d7f', '3fc907a5-71f1-122d-ec75-4f2a498f8f2b', 'a4fa2c7a-f461-7b5c-a0e6-55fe808b3b9a', '78f09d28-3eb5-f4d1-6a34-0878f43ca5ec', '4327616b-892e-be79-2f7a-7aca9ffe49db', '7c51a0b3-8b62-8fc9-d83b-50b2ea756e0d', '642f2b8f-0f92-7912-3c60-c09ec79b2c58', '297b261f-caca-3a4d-1b0c-d74b9e49e05c'], 'updated_user_ids': ['e8e54f69-7174-bc00-111b-b84460

In [ ]:
from utils.reset_system import reset_system
# reset_system()

In [ ]:
from qdrant_client import QdrantClient
from utils.qdrant_connection import client ,OWNER_PROFILES_COLLECTION


# Fetch all owner profiles (adjust limit if needed)
all_owners = client.scroll(
    collection_name=OWNER_PROFILES_COLLECTION,
    with_payload=True,
    limit=10000  # increase if you have more than 10k
)

updated_count = 0

for rec in all_owners[0]:  # scroll returns (records, next_page_offset)
    oid = str(rec.id)
    payload = rec.payload or {}
    email = payload.get("email")

    if isinstance(email, str) and "user" in email:
        new_email = email.replace("user", "owner")
        client.set_payload(
            collection_name=OWNER_PROFILES_COLLECTION,
            payload={"email": new_email},
            points=[oid],
        )
        updated_count += 1

print(f"✅ Updated {updated_count} owner emails")
